In [1]:
import requests
from bs4 import BeautifulSoup
import json
import time # Dùng để sleep chương trình
import pandas as pd # Dùng để đọc và hiển thị file csv (Pandas sẽ được học chi tiết ở buổi tới)
import datetime as dt # Dùng để xử lý dữ liệu thời gian
import re

In [30]:
Total_pages=10
req_headers = {
    'accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'accept-encoding':'gzip, deflate, sdch, br',
    'accept-language':'en-GB,en;q=0.8,en-US;q=0.6,ml;q=0.4',
    'cache-control':'max-age=0',
    'upgrade-insecure-requests':'1',
    'user-agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36'}
courses =pd.DataFrame()
Address_ =[]
Price_=[]
Bedrooms_=[]
Bathrooms_ =[]
LivingArea_ = []
Heating_ =[]
Basement_ =[]
Yearbuild_ =[]
Fireplace_ =[]
Garage_ =[]
with requests.Session() as s:
    page =1
    url = 'https://www.zillow.com/manchester-ct/sold/?searchQueryState=%7B"pagination"%3A%7B%7D%2C"usersSearchTerm"%3A"Manchester%2C%20CT"%2C"mapBounds"%3A%7B"west"%3A-72.62056917797851%2C"east"%3A-72.42659182202148%2C"south"%3A41.72559685174378%2C"north"%3A41.82852106786825%7D%2C"regionSelection"%3A%5B%7B"regionId"%3A398963%2C"regionType"%3A6%7D%5D%2C"isMapVisible"%3Atrue%2C"filterState"%3A%7B"fsba"%3A%7B"value"%3Afalse%7D%2C"fsbo"%3A%7B"value"%3Afalse%7D%2C"nc"%3A%7B"value"%3Afalse%7D%2C"fore"%3A%7B"value"%3Afalse%7D%2C"cmsn"%3A%7B"value"%3Afalse%7D%2C"auc"%3A%7B"value"%3Afalse%7D%2C"pmf"%3A%7B"value"%3Afalse%7D%2C"pf"%3A%7B"value"%3Afalse%7D%2C"rs"%3A%7B"value"%3Atrue%7D%2C"ah"%3A%7B"value"%3Atrue%7D%7D%2C"isListVisible"%3Atrue%2C"mapZoom"%3A13%7D'
    while(page < Total_pages):
        time.sleep(1)
        r = s.get(url, headers=req_headers)
        soup = BeautifulSoup(r.content, 'html.parser')
        detailLink = soup.find_all('a',{'class':'list-card-link list-card-link-top-margin list-card-img'})
        prices = soup.find_all('div',{'class':'list-card-price'})
        for price in prices:
            Price_.append(price.text)
        page+=1
        url ='https://www.zillow.com/manchester-nh/sold/' +str(page)+'_p/'
        for link in detailLink:
            address = link.text
            Address_.append(address)
            detail = s.get(link['href'],headers=req_headers)
            soup1 = BeautifulSoup(detail.content, 'html.parser')
            bbl = soup1.find_all('span',{'class':'ds-bed-bath-living-area'},limit =3) # bed-bad-livingarea
            bed_bad_livingarea =''
            for i in bbl:
                bed_bad_livingarea+= i.text +'\t'
            Bedrooms_.append(bed_bad_livingarea.split('\t')[0])
            Bathrooms_.append(bed_bad_livingarea.split('\t')[1])
            LivingArea_.append(bed_bad_livingarea.split('\t')[2])
            heating = soup1.find("span",string=re.compile("Heating:"))
            if heating is not None:
                heating = heating.find_next_sibling('span').text
                Heating_.append(heating)
            else:
                Heating_.append('None')
            basement = soup1.find("span",string=re.compile("Basement"))
            if basement is not None:
                if ':' in basement.find_next('span').text:
                    basement = basement.find_next('span').text.split(':')[1]
                    Basement_.append(basement)
                else:
                    Basement_.append('None')
            else:
                    Basement_.append('None')
            yearbuild = soup1.find("span",string=re.compile("Condition"))
            if yearbuild is not None:
                if ':' in yearbuild.find_next('span').text:
                    yearbuild = yearbuild.find_next('span').text.split(': ')[1]
                    Yearbuild_.append(yearbuild)
                else:
                    Yearbuild_.append('None')
            else:
                    Yearbuild_.append('None')

            fireplace = soup1.find("h5",string=re.compile("Property details"))
            if fireplace is not None:
                if 'Fireplace:' in fireplace.find_previous('span').text:
                    fireplace = fireplace.find_previous('span').text.split(': ')[1]
                    Fireplace_.append(fireplace)
                else:
                    Fireplace_.append('None')
            else:
                    Fireplace_.append('None')

            garage = soup1.find("span",string=re.compile("Property"))
            if garage is not None:
                if 'Garage spaces:' in garage.find_previous('span').text:
                    garage = garage.find_previous('span').text.split(': ')[1]
                    Garage_.append(garage)
                else:
                    Garage_.append('None')
            else:
                    Garage_.append('None')

courses['Address'] = Address_
courses['Bedrooms'] = Bedrooms_
courses['Bathrooms'] = Bathrooms_
courses['LivingArea'] = LivingArea_
courses['Heating'] = Heating_
courses['Basement'] = Basement_
courses['YearBuild'] = Yearbuild_
courses['Fireplace'] = Fireplace_
courses['Garage'] = Garage_
courses['Price'] = Price_
courses.to_csv('courses.csv')

<a aria-hidden="false" class="list-card-link list-card-link-top-margin list-card-img" href="https://www.zillow.com/homedetails/216-Walek-Farms-Rd-Manchester-CT-06040/174169099_zpid/" tabindex="-1"><img alt="216 Walek Farms Rd, Manchester, CT 06040" aria-hidden="false" class="" src="https://photos.zillowstatic.com/fp/9209ad426d18eeb14b72584b62a4c693-p_e.jpg"/></a>
3 bd	3 ba	2,556 sqft	
Baseboard, Gas
 Finished
1995
Yes
2
<a aria-hidden="false" class="list-card-link list-card-link-top-margin list-card-img" href="https://www.zillow.com/homedetails/39-S-Hawthorne-St-Manchester-CT-06040/62539673_zpid/" tabindex="-1"><img alt="39 S Hawthorne St, Manchester, CT 06040" aria-hidden="false" class="" src="https://photos.zillowstatic.com/fp/9e3038f6e687219d9015a31a5493c4d3-p_e.jpg"/></a>
3 bd	1 ba	1,080 sqft	
Forced air, Gas
 Partially finished
1945
0
<a aria-hidden="false" class="list-card-link list-card-link-top-margin list-card-img" href="https://www.zillow.com/homedetails/16-Saddle-Hill-Rd-Man

3 bd	3 ba	2,200 sqft	
Forced air, Gas
 Full, Full With Hatchway, Concrete Floor
Under Construction
2
<a class="list-card-link list-card-link-top-margin list-card-img" href="https://www.zillow.com/homedetails/43-Salem-Rd-Manchester-CT-06040/57744762_zpid/"></a>
3 bd	1 ba	1,338 sqft	
Forced air, Gas
 Partially Finished, Full With Hatchway, Concrete Floor, Hatchway Access
1941
Yes
1
<a class="list-card-link list-card-link-top-margin list-card-img" href="https://www.zillow.com/homedetails/512-Center-St-Manchester-CT-06040/174157139_zpid/"></a>
4 bd	1.5 ba	1,880 sqft	
Other
1927
Yes
0
<a class="list-card-link list-card-link-top-margin list-card-img" href="https://www.zillow.com/homedetails/22-Bolton-Center-Rd-Manchester-CT-06040/59077738_zpid/"></a>
3 bd	2 ba	1,200 sqft	
Baseboard, Stove, Oil
 Partially finished
1942
Yes
2
<a aria-hidden="false" class="list-card-link list-card-link-top-margin list-card-img" href="https://www.zillow.com/homedetails/50-Woodbridge-St-Manchester-CT-06042/590780

1 bd	1 ba	622 sqft	
Baseboard, Electric, Gas
 Yes
1978
0
<a class="list-card-link list-card-link-top-margin list-card-img" href="https://www.zillow.com/homedetails/182-Woodview-Way-Manchester-NH-03102/119151194_zpid/"></a>
2 bd	3 ba	1,594 sqft	
Forced air, Gas
2018
0
<a class="list-card-link list-card-link-top-margin list-card-img" href="https://www.zillow.com/homedetails/245-Edward-J-Roy-Dr-UNIT-310-Manchester-NH-03104/95348838_zpid/"></a>
2 bd	2 ba	1,081 sqft	
Forced air, Electric, Gas
 Yes
2001
1
<a aria-hidden="false" class="list-card-link list-card-link-top-margin list-card-img" href="https://www.zillow.com/homedetails/369-Hevey-St-Manchester-NH-03102/74280352_zpid/" tabindex="-1"></a>
4 bd	-- ba	1,785 sqft	
Other, Gas
 None
1910
4
<a class="list-card-link list-card-link-top-margin list-card-img" href="https://www.zillow.com/homedetails/225-Reed-St-Manchester-NH-03102/74295853_zpid/"></a>
2 bd	1 ba	988 sqft	
Other
1960
0
<a aria-hidden="false" class="list-card-link list-card-link-

3 bd	2.5 ba	2,726 sqft	
Other
1957
0
<a aria-hidden="false" class="list-card-link list-card-link-top-margin list-card-img" href="https://www.zillow.com/homedetails/77-Wayland-Ave-Manchester-NH-03109/74296241_zpid/" tabindex="-1"></a>
3 bd	2 ba	1,320 sqft	
Baseboard, Other, Gas
 Finished
1976
2
<a aria-hidden="false" class="list-card-link list-card-link-top-margin list-card-img" href="https://www.zillow.com/homedetails/77-Wayland-Ave-Manchester-NH-03109/74296241_zpid/" tabindex="-1"><img alt="77 Wayland Ave, Manchester, NH 03109" aria-hidden="false" class="" src="https://photos.zillowstatic.com/fp/c3c170841614a4debcf302eb00d9fd4b-p_e.jpg"/></a>
3 bd	2 ba	1,320 sqft	
Baseboard, Other, Gas
 Finished
1976
2
<a aria-hidden="false" class="list-card-link list-card-link-top-margin list-card-img" href="https://www.zillow.com/homedetails/33-S-Mammoth-Rd-Manchester-NH-03109/74324568_zpid/" tabindex="-1"><img alt="33 S Mammoth Rd, Manchester, NH 03109" aria-hidden="false" class="" src="https://pho

2 bd	3 ba	1,708 sqft	
Forced air, Electric, Gas, Oil
 Yes
2014
Yes
2
<a class="list-card-link list-card-link-top-margin list-card-img" href="https://www.zillow.com/homedetails/243-Union-St-Manchester-NH-03103/119152597_zpid/"></a>
12 bd	-- ba	4,302 sqft	
Baseboard, Gas
 Yes
1880
0
<a class="list-card-link list-card-link-top-margin list-card-img" href="https://www.zillow.com/homedetails/125-Stetson-St-Manchester-NH-03104/74322174_zpid/"></a>
3 bd	3 ba	2,316 sqft	
Electric, Gas, Oil
 Yes
1975
0
<a aria-hidden="false" class="list-card-link list-card-link-top-margin list-card-img" href="https://www.zillow.com/homedetails/153-Norton-Ave-Manchester-NH-03109/74289210_zpid/" tabindex="-1"></a>
4 bd	1 ba	1,104 sqft	
Baseboard, Oil
 Yes
1948
1
<a aria-hidden="false" class="list-card-link list-card-link-top-margin list-card-img" href="https://www.zillow.com/homedetails/7-S-Wilson-St-Manchester-NH-03103/74298353_zpid/" tabindex="-1"></a>
3 bd	2 ba	2,140 sqft	
Electric
 Yes
1973
1
<a aria-hidden="f

...

3 bd	3 ba	1,688 sqft	
Forced air, Electric, Gas
 Yes
1976
2
<a aria-hidden="false" class="list-card-link list-card-link-top-margin list-card-img" href="https://www.zillow.com/homedetails/190-Greenwood-Ct-Manchester-NH-03109/74329870_zpid/" tabindex="-1"></a>
4 bd	3 ba	3,022 sqft	
Forced air, Heat pump, Electric, Oil, Solar
 Yes
1972
Yes
1
<a aria-hidden="false" class="list-card-link list-card-link-top-margin list-card-img" href="https://www.zillow.com/homedetails/286-Lake-Ave-Manchester-NH-03103/74318096_zpid/" tabindex="-1"></a>
6 bd	-- ba	3,370 sqft	
Oil
 Yes
1900
0
<a aria-hidden="false" class="list-card-link list-card-link-top-margin list-card-img" href="https://www.zillow.com/homedetails/1569-Hall-St-Manchester-NH-03104/74328419_zpid/" tabindex="-1"></a>
4 bd	4 ba	3,267 sqft	
Forced air, Oil, Other, Propane / Butane
 Basement (not specified)
1984
Yes
8
<a aria-hidden="false" class="list-card-link list-card-link-top-margin list-card-img" href="https://www.zillow.com/homedetails/644

2 bd	2 ba	1,050 sqft	
Forced air, Gas
 None
2006
1
<a class="list-card-link list-card-link-top-margin list-card-img" href="https://www.zillow.com/homedetails/92-Gray-St-Manchester-NH-03103/74299383_zpid/"></a>
3 bd	2 ba	1,943 sqft	
Baseboard, Electric, Gas, Oil
 Yes
1956
1
<a aria-hidden="false" class="list-card-link list-card-link-top-margin list-card-img" href="https://www.zillow.com/homedetails/1602-Front-St-UNIT-6-Manchester-NH-03102/114777092_zpid/" tabindex="-1"></a>
2 bd	2 ba	1,972 sqft	
Forced air, Electric, Gas
 Yes
2004
1
<a aria-hidden="false" class="list-card-link list-card-link-top-margin list-card-img" href="https://www.zillow.com/homedetails/24-Charles-Chase-Way-Manchester-NH-03104/95348784_zpid/" tabindex="-1"><img alt="24 Charles Chase Way, Manchester, NH 03104" aria-hidden="false" class="" src="https://photos.zillowstatic.com/fp/75829538861663eb44211e259a845595-p_e.jpg"/></a>
2 bd	2 ba	2,001 sqft	
Forced air, Electric, Oil
 Yes
1988
Yes
2
<a aria-hidden="false" class=

3 bd	1 ba	1,354 sqft	
Other, Electric, Oil
 Yes
1950
1
<a aria-hidden="false" class="list-card-link list-card-link-top-margin list-card-img" href="https://www.zillow.com/homedetails/204-Boutwell-St-Manchester-NH-03102/74315862_zpid/" tabindex="-1"></a>
5 bd	2 ba	1,862 sqft	
Hot Water, Natural Gas
 Unfinished,Interior Entry
No
<a class="list-card-link list-card-link-top-margin list-card-img" href="https://www.zillow.com/homedetails/38-Brown-Ave-Manchester-NH-03101/119151154_zpid/"></a>
5 bd	-- ba	3,120 sqft	
Gas
 Yes
1980
0
<a aria-hidden="false" class="list-card-link list-card-link-top-margin list-card-img" href="https://www.zillow.com/homedetails/226-Fox-Hollow-Way-Manchester-NH-03104/95348719_zpid/" tabindex="-1"></a>
2 bd	2 ba	1,542 sqft	
Forced air, Gas
 Yes
1986
2
<a aria-hidden="false" class="list-card-link list-card-link-top-margin list-card-img" href="https://www.zillow.com/homedetails/610-Huse-Rd-Manchester-NH-03103/74328936_zpid/" tabindex="-1"></a>
3 bd	1 ba	1,728 sqft	
Forc

4 bd	-- ba	2,562 sqft	
Gas
 Yes
1875
2
<a aria-hidden="false" class="list-card-link list-card-link-top-margin list-card-img" href="https://www.zillow.com/homedetails/60-Charles-Chase-Way-Manchester-NH-03104/95349977_zpid/" tabindex="-1"><img alt="60 Charles Chase Way, Manchester, NH 03104" aria-hidden="false" class="" src="https://photos.zillowstatic.com/fp/87084d41d05a3d9d16848d8561b65ce9-p_e.jpg"/></a>
3 bd	3 ba	2,165 sqft	
Forced air, Electric, Oil
 Yes
1988
2
<a aria-hidden="false" class="list-card-link list-card-link-top-margin list-card-img" href="https://www.zillow.com/homedetails/881-Mammoth-Rd-405-Manchester-NH-03104/95350338_zpid/" tabindex="-1"><img alt="881 Mammoth Rd #405, Manchester, NH 03104" aria-hidden="false" class="" src="https://photos.zillowstatic.com/fp/dc831dad2acde54236db4e125e91a444-p_e.jpg"/></a>
2 bd	1 ba	1,100 sqft	
Baseboard, Gas
1985
Yes
2
<a aria-hidden="false" class="list-card-link list-card-link-top-margin list-card-img" href="https://www.zillow.com/hom

2 bd	2.5 ba	1,672 sqft	
Forced air
2017
0
<a class="list-card-link list-card-link-top-margin list-card-img" href="https://www.zillow.com/homedetails/3-Country-Club-Dr-APT-303-Manchester-NH-03102/119152157_zpid/"></a>
2 bd	2 ba	1,468 sqft	
Forced air, Electric, Gas
2005
Yes
1
<a class="list-card-link list-card-link-top-margin list-card-img" href="https://www.zillow.com/homedetails/357-Manchester-St-Manchester-NH-03103/114777144_zpid/"></a>
7 bd	3 ba	3,234 sqft	
Other
1903
5
<a class="list-card-link list-card-link-top-margin list-card-img" href="https://www.zillow.com/homedetails/470-Auburn-St-Manchester-NH-03103/74310522_zpid/"></a>
6 bd	-- ba	2,259 sqft	
Baseboard, Wall, Electric, Gas
 Yes
1900
0
<a class="list-card-link list-card-link-top-margin list-card-img" href="https://www.zillow.com/homedetails/574-Spruce-St-Manchester-NH-03103/74330320_zpid/"></a>
4 bd	-- ba	2,612 sqft	
Baseboard, Gas
 Yes
1900
0
<a aria-hidden="false" class="list-card-link list-card-link-top-margin list-card-i